# wps_bccaq
A process that runs the [bccaq.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/BCCAQ.R) function from ClimDown.

Bias Correction/Constructed Analogues with Quantile mapping reordering (BCCAQ) is composed of the following steps.
- Constructed Analogues (CA)
- Climate Imprint (CI)
- Quantile Delta Mapping (QDM)
- Rerank


In [1]:
from birdy import WPSClient
from netCDF4 import Dataset
from pkg_resources import resource_filename
from wps_tools.testing import get_target_url
from wps_tools.utils import copy_http_content
from tempfile import NamedTemporaryFile
from datetime import date
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")
#url = "http://docker-dev03.pcic.uvic.ca:30923/wps"

In [3]:
chickadee = WPSClient(url)

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.bccaq?

Signature:
chickadee.bccaq(
    gcm_file,
    obs_file,
    var=None,
    out_file=None,
    end_date=datetime.date(2005, 12, 31),
    num_cores='4',
    loglevel='INFO',
)
Docstring:
Full statistical downscaling of coarse scale global climate model (GCM) output to a fine spatial resolution

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
var : {'tasmax', 'tasmin', 'pr'}string
    Name of the NetCDF variable to downscale
end_date : date
    Defines the end of the calibration period
out_file : string
    Path to output file
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : Com

### Run the bccaq process

In [5]:
gcm_file = resource_filename("tests", "/data/tiny_gcm.nc"),
obs_file = resource_filename("tests", "/data/tiny_obs.nc"),
var = "tasmax"
end_date = date(1972, 12, 31)
out_file = "out.nc"
num_cores=1
output = chickadee.bccaq(
    gcm_file=gcm_file, 
    obs_file=obs_file, 
    var=var, 
    end_date=end_date,
    out_file=out_file,
    num_cores=num_cores
)
    
output_data = output.get()[0]

### Test for expected output

In [ ]:
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output_data, tmp_file))
    expected_data = Dataset(resource_filename("tests","/data/bccaq_expected_output.nc"))
    for key, value in expected_data.dimensions.items():
        assert str(output_data.dimensions[key]) == str(value)